#### Import Required Library

In [1]:
import cuxfilter
import cudf
from cuxfilter import DataFrame, themes, layouts
from cuxfilter.charts import bokeh
from bokeh.models import NumeralTickFormatter
from pyproj import Proj, Transformer
from pathlib import Path

#### insert csv data via Cudf

In [2]:
df = cudf.read_csv(r"/mnt/d/Learning/RAPIDS/JupyterCon_2020_RAPIDSViz/New folder/belgian_property_prices.csv")
df.head()

,as_built_plan,available_as_of,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,building_condition,co2_emission,...,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,housenumber,street,city,postal,state,lat,lng,price
0,0,After signing the deed,1,1,12,10,10,3,Good,9802,...,11,<NA>,30,Stationsstraat,Ronse,9600,Vlaams Gewest,50.743367,3.601306,275000
1,0,After signing the deed,1,1,15,11,11,4,Good,<NA>,...,<NA>,<NA>,52,Wingenesteenweg,Wingene,8750,Vlaams Gewest,51.042146,3.302600,430000
2,<NA>,After signing the deed,1,3,22,22,22,5,As new,<NA>,...,6,<NA>,157,Mechelsesteenweg,Antwerpen,2018,Vlaams Gewest,51.205031,4.410943,949000
3,0,<NA>,<NA>,3,33,30,23,5,As new,<NA>,...,19,<NA>,66,Noordzandstraat,Brugge,8000,Vlaams Gewest,51.206826,3.219186,2200000
4,0,After signing the deed,<NA>,<NA>,19,<NA>,<NA>,1,Just renovated,58,...,5,24263,15,Rue de l'Etoile,Wavre,1301,Région Wallonne,50.717062,4.577550,195000


In [3]:
df.describe()

/root/miniconda3/envs/rapids-23.10/lib/python3.10/site-packages/cudf/core/dataframe.py:5062: FutureWarning: `datetime_is_numeric` is deprecated. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(
/root/miniconda3/envs/rapids-23.10/lib/python3.10/site-packages/cudf/core/series.py:3334: FutureWarning: `datetime_is_numeric` is deprecated and will be removed in a future release. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(


,as_built_plan,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,co2_emission,cadastral_income,connection_to_sewer_network,...,surface_of_the_plot,tv_cable,tenement_building,toilets,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,postal,lat,lng,price
count,2389.000000,2083.0,4523.000000,3163.000000,3105.000000,2623.000000,4740.000000,1937.000000,3692.000000,2030.000000,...,4563.000000,1460.0,4723.000000,4348.000000,1990.000000,1.813000e+03,4737.000000,4737.000000,4737.000000,4.770000e+03
mean,0.062369,1.0,1.620385,18.351565,16.166506,17.777354,3.747257,536.673722,1363.668202,0.927586,...,1186.639711,1.0,0.110946,2.214811,18.709548,6.499582e+04,4880.245725,50.762168,4.456852,5.350881e+05
std,0.241876,0.0,1.109167,27.233866,36.545247,79.699364,1.528323,2193.643783,5672.238117,0.259236,...,4141.746595,0.0,0.314099,1.250844,59.314197,8.164299e+04,3045.197467,0.323770,0.744263,5.804791e+05
min,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,1.0,0.000000,1.000000,1.000000,1.000000e+00,1000.000000,49.559216,2.706054,2.900000e+04
25%,0.000000,1.0,1.000000,14.000000,12.000000,11.000000,3.000000,49.000000,567.000000,1.000000,...,202.500000,1.0,0.000000,1.000000,6.000000,3.789700e+04,1780.000000,50.615174,3.977402,2.650000e+05
50%,0.000000,1.0,1.000000,16.000000,14.000000,14.000000,3.000000,76.000000,929.000000,1.000000,...,466.000000,1.0,0.000000,2.000000,11.000000,5.561900e+04,4500.000000,50.805134,4.409118,3.950000e+05
75%,0.000000,1.0,2.000000,20.000000,17.000000,17.000000,4.000000,115.000000,1507.000000,1.000000,...,1000.000000,1.0,0.000000,3.000000,20.000000,8.115300e+04,8200.000000,50.964377,5.075631,5.839750e+05
max,1.000000,1.0,13.000000,1492.000000,1529.000000,2867.000000,25.000000,56557.000000,336965.000000,1.000000,...,110000.000000,1.0,1.000000,20.000000,1814.000000,2.980302e+06,9991.000000,51.356093,5.963238,8.350000e+06


#### Change null Value in several features to zero

In [37]:
df.fillna({'basement': 0, 'bathrooms': 0, 'bedroom_1_surface': 0, 'bedroom_2_surface': 0, 'bedroom_3_surface': 0, 'bedrooms': 0}, inplace=True)

In [5]:
df.head()

,as_built_plan,available_as_of,basement,bathrooms,bedroom_1_surface,bedroom_2_surface,bedroom_3_surface,bedrooms,building_condition,co2_emission,...,width_of_the_lot_on_the_street,yearly_theoretical_total_energy_consumption,housenumber,street,city,postal,state,lat,lng,price
0,0,After signing the deed,1,1,12,10,10,3,Good,9802,...,11,<NA>,30,Stationsstraat,Ronse,9600,Vlaams Gewest,50.743367,3.601306,275000
1,0,After signing the deed,1,1,15,11,11,4,Good,<NA>,...,<NA>,<NA>,52,Wingenesteenweg,Wingene,8750,Vlaams Gewest,51.042146,3.302600,430000
2,<NA>,After signing the deed,1,3,22,22,22,5,As new,<NA>,...,6,<NA>,157,Mechelsesteenweg,Antwerpen,2018,Vlaams Gewest,51.205031,4.410943,949000
3,0,<NA>,0,3,33,30,23,5,As new,<NA>,...,19,<NA>,66,Noordzandstraat,Brugge,8000,Vlaams Gewest,51.206826,3.219186,2200000
4,0,After signing the deed,0,0,19,0,0,1,Just renovated,58,...,5,24263,15,Rue de l'Etoile,Wavre,1301,Région Wallonne,50.717062,4.577550,195000


In [6]:
df.columns

Index(['as_built_plan', 'available_as_of', 'basement', 'bathrooms',
       'bedroom_1_surface', 'bedroom_2_surface', 'bedroom_3_surface',
       'bedrooms', 'building_condition', 'co2_emission', 'cadastral_income',
       'connection_to_sewer_network', 'construction_year',
       'covered_parking_spaces', 'dining_room', 'double_glazing',
       'energy_class', 'external_reference', 'flood_zone_type', 'furnished',
       'garden_surface', 'gas_water__electricity', 'heating_type',
       'kitchen_surface', 'kitchen_type', 'latest_land_use_designation',
       'living_area', 'living_room_surface', 'number_of_frontages', 'office',
       'outdoor_parking_spaces', 'planning_permission_obtained',
       'possible_priority_purchase_right', 'primary_energy_consumption',
       'proceedings_for_breach_of_planning_regulations',
       'reference_number_of_the_epc_report', 'street_frontage_width',
       'subdivision_permit', 'surface_of_the_plot', 'surroundings_type',
       'tv_cable', 'tenemen

In [7]:
df.dtypes

as_built_plan                                       int64
available_as_of                                    object
basement                                            int64
bathrooms                                           int64
bedroom_1_surface                                   int64
bedroom_2_surface                                   int64
bedroom_3_surface                                   int64
bedrooms                                            int64
building_condition                                 object
co2_emission                                        int64
cadastral_income                                    int64
connection_to_sewer_network                         int64
construction_year                                   int64
covered_parking_spaces                              int64
dining_room                                         int64
double_glazing                                      int64
energy_class                                       object
external_refer

#### Converting several features to Float32 to trial using linear regression

In [8]:
df = df.astype({'basement':'float32','bathrooms':'float32','bedroom_1_surface':'float32','bedroom_2_surface':'float32','bedroom_3_surface':'float32','bedrooms':'float32','price':'float32'})

#### Trial using Linear Regression to Predict Price

In [30]:
from cuml.datasets import make_regression
from cuml.model_selection import train_test_split
from cuml.linear_model import LinearRegression as cuLR
from sklearn.metrics import r2_score
import cupy

#### input X and Y Parameter also setup Train Test Split Data

In [31]:
X = df[['basement','bathrooms','bedroom_1_surface','bedroom_2_surface','bedroom_3_surface','bedrooms']]
y = df['price']
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split( X,
                                                                     y,
                                                                     train_size = 0.8,
                                                                     random_state = 20 )

In [32]:
cuml_reg_model = cuLR( fit_intercept = True,
                       normalize = True,
                       algorithm = 'eig' )

trained_LR = cuml_reg_model.fit( X_reg_train, y_reg_train )
cu_preds = trained_LR.predict( X_reg_test )

/root/miniconda3/envs/rapids-23.10/lib/python3.10/site-packages/cuml/internals/api_decorators.py:382: UserWarning: Starting from version 23.08, the new 'copy_X' parameter defaults to 'True', ensuring a copy of X is created after passing it to fit(), preventing any changes to the input, but with increased memory usage. This represents a change in behavior from previous versions. With `copy_X=False` a copy might still be created if necessary. Explicitly set 'copy_X' to either True or False to suppress this warning.
  return init_func(self, *args, **filtered_kwargs)


In [33]:
cu_preds.head()

463     232869.265625
1368    408151.250000
3446    622691.375000
4496    537407.750000
515     418464.906250
dtype: float32

In [36]:
y_reg_test.head()

463     126000.0
1368    155000.0
3446    389000.0
4496    750000.0
515     335000.0
Name: price, dtype: float32

##### 'basement','bathrooms','bedroom_1_surface','bedroom_2_surface','bedroom_3_surface',& 'bedrooms' feature still not enough to predict home prices. will try to include other features in next trial


#### Insert Dataframe into cuXFilter

In [38]:
cux_df = cuxfilter.DataFrame.from_dataframe(df)

#### Create Bar Chart

In [39]:
bar_chart_1 = bokeh.bar(
    'city', 'price', data_points=5, add_interaction=False
)

#### Create Dashboard From Bar Chart

In [40]:
d = cux_df.dashboard([bar_chart_1])

#### Display Example Chart

In [41]:
d.app(sidebar_width=200, width=1000, height=450)

GridSpec(ncols=12, nrows=1)
    [0] GridSpec(height=450, ncols=12, nrows=5, sizing_mode='fixed', width=1000)
        [0] HoloViews(DynamicMap, height=450, sizing_mode='stretch_both', width=1000)
    [1] WidgetBox(styles={'border-color': '...})
        [0] Number(css_classes=['indicator'], default_color='#2B2B2B', font_size='18pt', format='{value:,}', name='Datapoints Selected', sizing_mode='stretch_width', title_size='14pt', value=4770)
        [1] Progress(sizing_mode='stretch_width', styles={'--success-bg-color': '...}, value=100)